In [2]:
!apt-get update
!apt-get install -y g++ openjdk-8-jdk python3-dev   # 필요시
!pip install pandas scikit-learn xgboost transformers sentence-transformers bertopic joblib umap

# 코랩에서:
from google.colab import drive
drive.mount('/content/drive')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pa

In [ ]:
!apt-get update
!pip install "jpype1==1.5.0" konlpy
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

from konlpy.tag import Komoran

komoran = Komoran()
print(komoran.morphs("JPype와 Komoran 테스트 중입니다."))

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
['JPype', '와',

In [ ]:
# %% 전역상수 선언
import re
import pathlib
import torch
from konlpy.tag import Komoran
from sentence_transformers import SentenceTransformer

PROJECT_ROOT = pathlib.Path('/content/drive/MyDrive/IBAS')
DATA_ROOT = PROJECT_ROOT / 'DataSet_DATAUP2025'
RAW_DIR = DATA_ROOT / 'RawData'
CLEANED_DIR = DATA_ROOT / 'Cleaned'
MODELS_DIR = DATA_ROOT / 'Models'
SCORED_DIR = DATA_ROOT / 'Scored'
LABELED_DIR = DATA_ROOT / 'StrongLabeled'

MIN_REVIEW_LENGTH = 10
KAKAO_TO_NAVER_COLS = {
    "가게이름": "가게이름",          # 동일 의미
    "카테고리": "카테고리",          # 동일 의미
    "전체평점": "전체평점",          # 동일 의미
    "평점건수": "방문자리뷰",        # Kakao 평점건수 → Naver 방문자리뷰(리뷰 수)
    "리뷰작성자": "리뷰작성자",      # 동일 의미
    "작성자리뷰작성수": "리뷰작성수",  # Kakao 작성자리뷰작성수 → Naver 리뷰작성수
    "리뷰내용": "리뷰내용",          # 동일 의미
    "리뷰작성일": "방문시간",        # Kakao 리뷰작성일 → Naver 방문시간(시점)
    # Kakao의 '작성자평균평점', '리뷰평점' 등은 Naver에 대응 컬럼이 없으므로 매핑하지 않음
}


ALLOWED_PATTERN = re.compile(r'[^가-힣ㄱ-ㅎㅏ-ㅣA-Za-z0-9\\s.,?!]+') # 형태소 분석 모델에서 처리하지 못 하는 문자열
PUNCT_SKIP_PATTERN = re.compile(r"^[^\w가-힣]+$")
MORPH_ANALYZER = Komoran()
KEYWORD_POS_TAGS = {"NNG", "NNP", "VA", "MAG", "XR"}
SBERT_MODEL = SentenceTransformer('jhgan/ko-sbert-sts',
            device = 'cuda' if torch.cuda.is_available() else 'cpu') # SBERT 모델

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# %% 전용 소규모 한국어 사전
MENU_TERMS = {
    # 고기/한식 메인
    "고기", "삼겹살", "목살", "갈비", "갈비탕", "돼지", "돼지고기",
    "소고기", "소금구이", "갈매기살", "족발", "보쌈", "등심", "대패",
    "오리", "통닭", "닭갈비", "닭발", "닭볶음탕", "대창", "곱창",
    "돼지갈비", "육회", "육전", "불고기", "육개장",

    # 탕/찌개/국/국밥
    "국물", "감자탕", "된장찌개", "김치찌개", "해장국", "전골", "곱창전골",
    "곰탕", "설렁탕", "순두부찌개", "순두부", "무침", "탕수육", "감자전",
    "해물", "해산물", "게장", "꽃게", "해물탕",

    # 면/분식류
    "떡볶이", "쫄면", "우동", "라멘", "라면", "국수", "칼국수",
    "냉면", "막국수", "쌀국수", "짬뽕", "짜장면", "비빔국수",
    "순대", "순대국", "김밥", "만두", "오뎅", "어묵", "분식", "분식집",

    # 일식/양식/기타
    "초밥", "사시미", "소바", "덮밥", "돈가스", "파스타", "스파게티",
    "스테이크", "피자", "버거", "햄버거", "토스트", "샌드위치",
    "브런치", "와인", "카레", "팟타이", "보리밥",

    # 사이드/디저트
    "반찬", "밑반찬", "샐러드", "계란찜", "계란말이", "아이스크림",
    "디저트", "와플", "빙수", "팥빙수", "과일", "케이크",
    "치즈볼", "에이드", "커피", "아메리카노", "음료", "주스",

    # 재료/토핑
    "김치", "새우", "연어", "버섯", "마늘", "숙주", "명란", "야채",
    "채소", "베이컨", "고구마", "계란", "콩나물", "양파", "미나리",
}

TASTE_TERMS = {
    "맛있", "맛나", "맛없", "고소", "달달", "달콤", "짭짤",
    "맵", "매콤", "얼큰", "담백", "진한", "싱겁", "느끼",
    "깔끔", "부드럽", "쫄깃", "바삭", "바삭바삭", "촉촉",
    "싱싱", "신선", "풍부", "풍미", "담백", "진리", "꿀맛",
}

SERVICE_TERMS = {
    "친절", "응대", "서비스", "사장", "사장님", "직원", "알바",
    "주방", "센스", "정성", "신경", "배려", "설명",
}

AMBIENCE_TERMS = {
    "분위기", "인테리어", "음악", "조명", "노래", "감성",
    "깔끔", "청결", "위생", "쾌적", "아늑", "조용", "시끄럽",
    "자리", "좌석", "테이블", "내부", "외부", "창문", "창가",
}

PRICE_TERMS = {
    "가격", "가성비", "저렴", "비싸", "합리", "혜자",
    "만원", "천원", "물가", "매우", "합리", "할인",
}

QUANTITY_TERMS = {
    "양도", "양이", "양은", "양", "푸짐", "넉넉", "배부르",
    "배불리", "든든", "가득", "한입", "가득",
}

SITUATION_TERMS = {
    # 관계/타깃
    "친구", "가족", "아이", "아이들", "부모", "엄마", "아빠",
    "남편", "남자친구", "여자친구", "연인", "부부", "직장인",
    "대학생", "학생", "혼자", "혼밥", "커플",

    # 용도/상황
    "데이트", "회식", "모임", "단체", "회식", "외식", "야식",
    "점심", "점심시간", "저녁", "브런치", "주말", "방학",

    # 지역/상권
    "인하대", "인하대학교", "인하대역", "용현동", "동네", "대학가",
    "인천", "송도", "서울", "본점", "골목",
}

DOMAIN_KEYWORDS = (
    MENU_TERMS | TASTE_TERMS | SERVICE_TERMS |
    AMBIENCE_TERMS | PRICE_TERMS | QUANTITY_TERMS | SITUATION_TERMS
)
DOMAIN_KEYWORDS_LEN3 = {w for w in DOMAIN_KEYWORDS if len(w) >= 3}

DOMAIN_STOPWORDS = {
    # 강도/상투 부사
    "정말", "진짜", "너무", "너무너무", "굉장히", "매우", "엄청",
    "엄청나", "완전", "진심", "항상", "맨날", "매번", "종종",
    "조금", "약간", "적당히", "넘넘", "상당히", "꽤",

    # 시간/상황 표현 (형식적)
    "오늘", "어제", "이번", "저번", "다음", "다음번", "주말",
    "요즘", "최근", "방학", "날씨", "하루", "일주일",

    # 리뷰/행동 표현
    "방문", "이용", "선택", "예약", "결제", "검색", "리뷰", "후기",
    "사용", "작성", "이렇", "이예", "라고", "해서", "벌써",

    # 감정 표현 중 정보량 적은 것
    "행복", "행복합니다", "감사", "감동", "재밌", "재미",

    # 기타 포괄적인 일반어
    "그냥", "솔직히", "정도", "부분", "전체", "일단",
}

In [ ]:
# %% 자의적 리뷰 데이터 준비
import numpy as np
import pandas as pd
from collections import Counter

def align_kakao_to_naver(
    naver_df: pd.DataFrame,
    kakao_df: pd.DataFrame,
    col_map: dict,
) -> pd.DataFrame:
    """
    Kakao 리뷰를 Naver 스키마에 맞게 매핑한 뒤 두 데이터프레임을 수직 결합한다.
    - col_map에 정의되지 않은 Kakao 컬럼은 무시
    - Naver에 없는 컬럼은 생성하지 않음
    """
    effective_map = {
        k: v
        for k, v in col_map.items()
        if (k in kakao_df.columns) and (v in naver_df.columns)
    }

    kakao_aligned = kakao_df[list(effective_map.keys())].rename(columns=effective_map)

    # Naver에는 있는데 Kakao에는 없는 컬럼은 NaN으로 채움
    for col in naver_df.columns:
        if col not in kakao_aligned.columns:
            kakao_aligned[col] = pd.NA

    kakao_aligned = kakao_aligned[naver_df.columns]

    merged = pd.concat([naver_df, kakao_aligned], ignore_index=True)
    return merged

def filter_voluntary_reviews(
    df: pd.DataFrame,
    text_col: str = "리뷰내용",
    ad_pred_col: str = "ad_pred",   # 0=자의적, 1=광고
) -> pd.DataFrame:
    """
    광고성 예측 라벨을 이용해 자의적 리뷰 코퍼스를 구성한다.
    - ad_pred == 0인 리뷰만 사용
    - 텍스트 길이가 MIN_REVIEW_LENGTH 미만인 리뷰는 제거
    """
    df = df.copy()
    df[text_col] = df[text_col].fillna("")

    mask = (df[ad_pred_col] == 0) & (df[text_col].str.len() >= MIN_REVIEW_LENGTH)
    df = df[mask].reset_index(drop=True)
    return df

naver_scored = pd.read_csv(SCORED_DIR / "Naver_Reviews_AdScored.csv", encoding="utf-8-sig")
kakao_scored = pd.read_csv(SCORED_DIR / "Kakao_Reviews_AdScored.csv", encoding="utf-8-sig")

merged_reviews = align_kakao_to_naver(naver_scored, kakao_scored, KAKAO_TO_NAVER_COLS)
print(merged_reviews['ad_pred'].value_counts())
reviews_corpus = filter_voluntary_reviews(merged_reviews, text_col="리뷰내용", ad_pred_col="ad_pred")
print("원본 리뷰 수:", len(merged_reviews))
print("자의적 리뷰 수:", len(reviews_corpus))

/tmp/ipython-input-3690851382.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat([naver_df, kakao_aligned], ignore_index=True)


ad_pred
0    22729
1     6500
Name: count, dtype: int64
원본 리뷰 수: 31070
자의적 리뷰 수: 13975


In [ ]:
# %% SBERT + BERTopic 토픽 모델링 & 가게별 키워드/바이그램 추출
import re
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import silhouette_score
from konlpy.tag import Komoran
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer


# ---------------------------------------------------
# 공통 유틸
# ---------------------------------------------------

def clean_text(text: str) -> str:
    """이모지/제어문자 제거 및 공백 정리."""
    if not isinstance(text, str):
        text = str(text)
    text = ALLOWED_PATTERN.sub(" ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def is_one_char_diff(a: str, b: str) -> bool:
    """길이가 같을 때 서로 다른 글자가 정확히 1개면 True."""
    if len(a) != len(b):
        return False
    diff = 0
    for ca, cb in zip(a, b):
        if ca != cb:
            diff += 1
            if diff > 1:
                return False
    return diff == 1


def correct_with_domain_dict(lemma: str) -> str:
    """
    lemma가 도메인 사전 단어와 한 글자만 다르면
    그 도메인 단어로 교정해 반환.
    여러 후보가 있으면 그대로 둔다(모호성 회피).
    """
    if len(lemma) < 3:
        return lemma

    candidates = [
        w for w in DOMAIN_KEYWORDS_LEN3
        if len(w) == len(lemma) and is_one_char_diff(lemma, w)
    ]

    if len(candidates) == 1:
        return candidates[0]
    else:
        return lemma

def is_valid_bigram(w1: str, w2: str) -> bool:
    """
    DOMAIN_KEYWORDS 구조를 고려해 '말이 되는' 바이그램만 허용.
    - 같은 단어 두 번 반복: 금지 (맛있 맛있 등)
    - 역할 조합이 의미 있는 경우만 True
    """
    # 1) 동일 단어 반복 금지
    if w1 == w2:
        return False

    # 2) 각 단어의 타입判定
    def word_type(w):
        if w in MENU_TERMS:
            return "menu"
        if w in TASTE_TERMS:
            return "taste"
        if w in SERVICE_TERMS:
            return "service"
        if w in AMBIENCE_TERMS:
            return "ambience"
        if w in PRICE_TERMS:
            return "price"
        if w in QUANTITY_TERMS:
            return "quantity"
        if w in SITUATION_TERMS:
            return "situation"
        return "other"

    t1 = word_type(w1)
    t2 = word_type(w2)

    # 3) 허용할 조합 정의
    # 메뉴 + 맛/가격/양
    if t1 == "menu" and t2 in {"taste", "price", "quantity"}:
        return True
    # 맛/가격/양 + 메뉴  (국물 떡볶이 이런 건 애매하지만 허용 여부는 선택)
    if t1 in {"taste", "price", "quantity"} and t2 == "menu":
        return True

    # 상황 + 메뉴/분위기 (인하대 떡볶이, 데이트 술집 등)
    if t1 == "situation" and t2 in {"menu", "ambience"}:
        return True

    # 기타: 특별히 허용하고 싶은 패턴이 있으면 추가
    # 예: '가성비 좋', '분위기 좋' 등 taste형 용언과의 조합
    if w2 in {"좋", "괜찮", "훌륭"} and t1 in {"menu", "ambience", "price"}:
        return True

    return False


def tokenize_for_domain(text: str):
    """
    하나의 문장을 도메인 사전/불용어/형태소 필터링을 통과한 토큰 리스트로 변환.
    - KEYWORD_POS_TAGS: 사용할 품사 (NNG/NNP/VA/MAG/XR 등)
    - DOMAIN_STOPWORDS, VERB_STOP_LEMMAS: 도메인/동사 불용어
    - DOMAIN_KEYWORDS: 식당 도메인 핵심 키워드 사전
    """
    text = clean_text(text)
    if not text:
        return []

    try:
        morphs = MORPH_ANALYZER.pos(text)
    except Exception:
        return []

    tokens = []
    for lemma, pos in morphs:
        # 1) 품사 필터
        if pos not in KEYWORD_POS_TAGS:
            continue

        # 2) 숫자 제거
        if lemma.isdigit():
            continue

        # 3) 기호/이모티콘 제거
        if not re.search(r"[가-힣A-Za-z0-9]", lemma):
            continue

        # 4) 길이 1 제거
        if len(lemma) <= 1:
            continue

        # 5) 도메인/동사 불용어 제거
        if lemma in DOMAIN_STOPWORDS:
            continue

        # 6) 도메인 사전 기반 오탈자 교정
        lemma = correct_with_domain_dict(lemma)

        # 7) 도메인 키워드에 없는 단어는 제외
        if lemma not in DOMAIN_KEYWORDS:
            continue

        tokens.append(lemma)

    return tokens


# ---------------------------------------------------
# SBERT + BERTopic (리뷰 수준 토픽)
# ---------------------------------------------------

def build_review_topics(
    df: pd.DataFrame,
    text_col: str = "리뷰내용",
    sbert_model: SentenceTransformer = None,
    batch_size: int = 64,
):
    if sbert_model is None:
        sbert_model = SBERT_MODEL

    df = df.copy()
    df[text_col] = df[text_col].fillna("").apply(clean_text)
    docs = df[text_col].tolist()

    # 1) SBERT 임베딩
    embeddings = sbert_model.encode(
        docs,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
    )
    embeddings = np.nan_to_num(embeddings, nan=0.0, posinf=1e3, neginf=-1e3)

    # 2) BERTopic용 CountVectorizer (pruning을 최대한 느슨하게)
    vectorizer_model = CountVectorizer(
        token_pattern=r"(?u)\b[가-힣A-Za-z0-9]+\b",
        max_df=0.98,      # 아주 자주 나오는 단어 일부만 제거
        min_df=1,         # 토픽 문서 수에 상관없이 항상 최소 1개 문서에서만 나와도 허용
        ngram_range=(1, 1),   # 우선 1-gram만 사용 (2-gram은 Komoran 바이그램으로 보강)
        max_features=20_000,
    )

    topic_model = BERTopic(
        vectorizer_model=vectorizer_model,
        calculate_probabilities=True,
        verbose=True,
    )

    topics, probs = topic_model.fit_transform(docs, embeddings)
    print("고유 토픽 수:", len(set(topics)))

    df_topics = df.copy()
    df_topics["topic"] = topics
    df_topics["topic_prob"] = probs.max(axis=1)

    return df_topics, topic_model, embeddings


# ---------------------------------------------------
# 전역 바이그램 사전 생성
# ---------------------------------------------------

def build_global_bigrams(
    corpus_df: pd.DataFrame,
    text_col: str = "리뷰내용",
    min_count: int = 20,
):
    bigram_counter = Counter()

    for text in corpus_df[text_col].fillna(""):
        tokens = tokenize_for_domain(text)
        for i in range(len(tokens) - 1):
            w1, w2 = tokens[i], tokens[i+1]
            # 말이 되는 바이그램만 카운트
            if not is_valid_bigram(w1, w2):
                continue
            bg = f"{w1} {w2}"
            bigram_counter[bg] += 1

    global_bigrams = {
        bg for bg, c in bigram_counter.items() if c >= min_count
    }
    return global_bigrams


# ---------------------------------------------------
# 가게별: 3개 대표 바이그램 + 12개 단일 키워드 추출
# ---------------------------------------------------

def extract_keywords_and_bigrams_for_store(
    texts,
    global_bigrams,
    top_unigrams: int = 12,
    top_bigrams: int = 3,
):
    unigram_counter = Counter()
    bigram_counter = Counter()

    for text in texts:
        tokens = tokenize_for_domain(text)
        if not tokens:
            continue

        for t in tokens:
            unigram_counter[t] += 1

        for i in range(len(tokens) - 1):
            w1, w2 = tokens[i], tokens[i+1]
            if not is_valid_bigram(w1, w2):
                continue
            bg = f"{w1} {w2}"
            if bg in global_bigrams:
                bigram_counter[bg] += 1

    top_bigram_list = [bg for bg, _ in bigram_counter.most_common(top_bigrams)]
    top_unigram_list = [w for w, _ in unigram_counter.most_common(top_unigrams)]
    combined_keywords = top_bigram_list + top_unigram_list
    return combined_keywords, top_bigram_list, top_unigram_list


def build_store_profiles(
    df_topics: pd.DataFrame,
    topic_model: BERTopic,
    global_bigrams,
    store_col: str = "가게이름",
    text_col: str = "리뷰내용",
) -> pd.DataFrame:
    """
    리뷰별 topic 정보를 가게 단위로 집계해:
    - topic_distribution: {topic_id: 비율}
    - keywords: 대표 bigram 3개 + 단일 키워드 12개
    - bigrams/unigrams: 필요 시 별도 컬럼으로도 저장
    """
    rows = []

    for store, group in df_topics.groupby(store_col):
        # 1) 해당 가게의 토픽 분포
        topic_counts = group["topic"].value_counts(normalize=True)

        # 2) Komoran 기반 토큰에서 바이그램 + 단일 키워드 추출
        texts = group[text_col].tolist()
        combined_keywords, top_bigrams, top_unigrams = (
            extract_keywords_and_bigrams_for_store(
                texts,
                global_bigrams=global_bigrams,
                top_unigrams=12,
                top_bigrams=3,
            )
        )

        rows.append(
            {
                store_col: store,
                "topic_distribution": topic_counts.to_dict(),
                "keywords": combined_keywords,   # [3 bigrams + 12 unigrams]
                "bigrams": top_bigrams,          # 선택: 별도로 보고 싶을 때
                "unigrams": top_unigrams,        # 선택: 별도로 보고 싶을 때
            }
        )

    store_profiles = pd.DataFrame(rows)
    return store_profiles


# 전역 바이그램 사전 생성
global_bigrams = build_global_bigrams(
    corpus_df=reviews_corpus,
    text_col="리뷰내용",
    min_count=20,   # 코퍼스 크기에 따라 10~30 사이에서 조정
)

# 리뷰 수준 토픽 모델링
review_topics, topic_model, embeddings = build_review_topics(
    reviews_corpus,
    text_col="리뷰내용",
)

# 가게별 프로파일 (3개 바이그램 + 12개 단일 키워드)
store_profiles = build_store_profiles(
    review_topics,
    topic_model,
    global_bigrams=global_bigrams,
    store_col="가게이름",
    text_col="리뷰내용",
)
print(store_profiles.head(15))


Batches:   0%|          | 0/219 [00:00<?, ?it/s]

2025-11-17 15:27:14,113 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-17 15:27:14,359 - BERTopic - Dimensionality - Completed ✓
2025-11-17 15:27:14,362 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.12/dist-packages/hdbscan/prediction.py:663: RuntimeWarning: invalid value encountered in scalar divide
  in_cluster_probs = all_points_prob_in_some_cluster(
2025-11-17 15:27:16,239 - BERTopic - Cluster - Completed ✓
2025-11-17 15:27:16,247 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-17 15:27:16,341 - BERTopic - Representation - Completed ✓


고유 토픽 수: 19
             가게이름                                 topic_distribution  \
0           153횟집    {0: 0.6666666666666666, -1: 0.3333333333333333}   
1            29포차    {-1: 0.6842105263157895, 0: 0.3157894736842105}   
2      87닭강정 인하대점    {-1: 0.6666666666666666, 0: 0.3333333333333333}   
3    BBQ치킨 인하대후문점                                           {0: 1.0}   
4     BitterSweet    {-1: 0.7142857142857143, 0: 0.2857142857142857}   
5   SHOHONA KEBAB                                           {0: 1.0}   
6            UZ케밥                                           {0: 1.0}   
7             가메이  {-1: 0.6181818181818182, 0: 0.3454545454545454...   
8            가시버시  {-1: 0.46153846153846156, 0: 0.384615384615384...   
9      감탄떡볶이 인하대점   {-1: 0.8461538461538461, 0: 0.15384615384615385}   
10          경동삼겹살   {-1: 0.6538461538461539, 0: 0.34615384615384615}   
11         계속 쪼르면  {-1: 0.6633333333333333, 0: 0.3033333333333333...   
12           고양이눈  {-1: 0.6086956521739131, 0: 0.347

In [ ]:
# %% 전용 소규모 한국어 사전 제작, 키워드 추출 후 실행
def build_raw_vocab(corpus_df, text_col="리뷰내용"):
    counter = Counter()
    for text in corpus_df[text_col].fillna(""):
        for lemma, pos in MORPH_ANALYZER.pos(clean_text(text)):
            if pos not in KEYWORD_POS_TAGS:
                continue
            if len(lemma) <= 1:
                continue
            if lemma.isdigit():
                continue
            if not re.search(r"[가-힣A-Za-z0-9]", lemma):
                continue
            counter[lemma] += 1
    return counter

raw_vocab = build_raw_vocab(reviews_corpus)
common_terms = [w for w, c in raw_vocab.most_common(5000)]

with open(DATA_ROOT / 'Common_Terms.txt', 'w', encoding='utf-8') as f:
    # ' / '로 연결하여 TXT 파일로 쓰기
    f.write(' / '.join(common_terms))

In [ ]:
# %% 리뷰 내용 유사성 기반 가게 군집화
import re
import hdbscan
import umap.umap_ as umap
import numpy as np
import pandas as pd
from collections import Counter
from konlpy.tag import Komoran
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer


def build_store_embeddings(
    review_topics: pd.DataFrame,
    embeddings: np.ndarray,
    store_col: str = "가게이름",
):
    """
    리뷰 수준 SBERT 임베딩을 가게 단위로 평균하여 store-level embedding을 만든다.
    """
    review_topics = review_topics.reset_index(drop=True)
    assert len(review_topics) == len(embeddings)

    store_embeds = {}
    for store, group in review_topics.groupby(store_col):
        idx = group.index.values
        store_vec = embeddings[idx].mean(axis=0)
        store_embeds[store] = store_vec

    stores = list(store_embeds.keys())
    X = np.vstack([store_embeds[s] for s in stores])
    store_emb_df = pd.DataFrame(X)
    store_emb_df[store_col] = stores

    return store_emb_df


# %% 5) 클러스터링 & 클러스터 프로파일링
def summarize_cluster_keywords(store_clusters, cluster_id, top_k=10):
    rows = store_clusters[store_clusters["cluster_id"] == cluster_id]

    bigram_counter = Counter()
    unigram_counter = Counter()

    for _, row in rows.iterrows():
        for bg in row["bigrams"]:
            bigram_counter[bg] += 1
        for ug in row["unigrams"]:
            unigram_counter[ug] += 1

    top_bigrams = [bg for bg, _ in bigram_counter.most_common(top_k)]
    top_unigrams = [ug for ug, _ in unigram_counter.most_common(top_k)]

    return top_bigrams, top_unigrams


def summarize_cluster_topics(store_clusters, cluster_id):
    rows = store_clusters[store_clusters["cluster_id"] == cluster_id]
    agg = Counter()
    for dist in rows["topic_distribution"]:
        for t, p in dist.items():
            agg[t] += p
    total = sum(agg.values())
    if total > 0:
        for t in agg:
            agg[t] /= total
    return dict(agg)


# %% 전체 실행 파이프라인
store_profiles["n_bigrams"] = store_profiles["bigrams"].apply(len)
store_profiles["n_unigrams"] = store_profiles["unigrams"].apply(len)
cluster_candidates = store_profiles[
    (store_profiles["n_bigrams"] >= 2) &
    (store_profiles["n_unigrams"] >= 5)
].copy()

# 가게 단위 SBERT 임베딩
store_emb_df = build_store_embeddings(
    review_topics,
    embeddings,
    store_col="가게이름",
)

X_store = store_emb_df.merge(
    cluster_candidates[["가게이름"]],
    on="가게이름",
    how="inner"
)
store_names = X_store["가게이름"].tolist()
X_vec = X_store.drop(columns=["가게이름"]).to_numpy()

umap_model = umap.UMAP(
        n_neighbors=15,        # 지역 구조 강조 (10~50 범위에서 튜닝)
        n_components=15,
        min_dist=0.0,          # 클러스터링에 적합하게 점들을 더 조밀하게 배치
        metric="cosine",       # SBERT 임베딩에는 cosine이 일반적
        random_state=42,
    )
X_umap = umap_model.fit_transform(X_vec)   # shape: [n_candidates, 15]

# 8) HDBSCAN 군집화
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=5,          # 최소 클러스터 크기 (데이터 보고 5~20 사이 튜닝)
    min_samples=2,               # 노이즈 비율 줄이고 싶으면 1~5 정도로 낮춰보기
    metric="euclidean",          # UMAP 출력에는 euclidean이 기본
    cluster_selection_method="eom",
)
cluster_labels = clusterer.fit_predict(X_umap)

# HDBSCAN 결과: cluster_labels, UMAP 결과: X_umap 라고 가정
labels = cluster_labels

# 1) 노이즈(-1) 제외한 포인트만 사용
mask = labels != -1
X_umap_valid = X_umap[mask]
labels_valid = labels[mask]

n_clusters_valid = len(set(labels_valid))
n_points_valid = len(labels_valid)

print("유효 포인트 수(노이즈 제외):", n_points_valid)
print("유효 클러스터 수(노이즈 제외):", n_clusters_valid)

sil_score = None
if n_clusters_valid >= 2 and n_points_valid >= 10:
    sil_score = silhouette_score(X_umap_valid, labels_valid, metric="euclidean")
    print("실루엣 스코어 (UMAP 공간, 노이즈 제외):", sil_score)
else:
    print("클러스터가 2개 미만이거나 포인트 수가 너무 적어 실루엣 스코어를 계산하지 않습니다.")

# 9) 결과 결합
cluster_result = pd.DataFrame(
    {"가게이름": store_names, "cluster_id": cluster_labels}
)

store_clusters = cluster_result.merge(
    store_profiles[["가게이름", "bigrams", "unigrams", "topic_distribution"]],
    on="가게이름",
    how="left"
)

print(store_clusters.head())
print(store_clusters['cluster_id'].value_counts())

유효 포인트 수(노이즈 제외): 126
유효 클러스터 수(노이즈 제외): 12
실루엣 스코어 (UMAP 공간, 노이즈 제외): 0.5334932
         가게이름  cluster_id                  bigrams  \
0       153횟집           1    [초밥 맛있, 맛있 새우, 새우 맛있]   
1         가메이           3    [연어 맛있, 맛있 연어, 김치 맛있]   
2        가시버시           7    [맛있 김치, 국물 맛있, 김치 맛있]   
3  감탄떡볶이 인하대점           4  [떡볶이 맛있, 맛있 떡볶이, 국물 맛있]   
4       경동삼겹살          -1   [반찬 맛있, 삼겹살 맛있, 맛있 반찬]   

                                            unigrams  \
0  [맛있, 초밥, 새우, 사장, 친절, 점심, 가격, 싱싱, 아이, 국물, 인하대, 소바]   
1  [맛있, 연어, 새우, 인하대, 덮밥, 우동, 양이, 양도, 가격, 친절, 신선, ...   
2  [맛있, 김치, 가격, 진한, 국물, 깔끔, 분위기, 칼국수, 육회, 양도, 친구,...   
3  [맛있, 떡볶이, 친절, 순대, 깔끔, 맛나, 국물, 사장, 바삭, 분식, 테이블,...   
4  [반찬, 맛있, 인하대, 사장, 양도, 가격, 친절, 삼겹살, 채소, 냉면, 푸짐,...   

                                  topic_distribution  
0    {0: 0.6666666666666666, -1: 0.3333333333333333}  
1  {-1: 0.6181818181818182, 0: 0.3454545454545454...  
2  {-1: 0.46153846153846156, 0: 0.384615384615384...  
3   {-1: 0.8461538461538461, 0: 0.153

In [ ]:
# %% 가게 군집별 키워드/바이그램 추출
from collections import Counter

def extract_keywords_and_bigrams_for_cluster(
    texts,
    global_bigrams,
    n_top_unigrams: int = 12,
    n_top_bigrams: int = 3,
):
    unigram_counter = Counter()
    bigram_counter = Counter()

    for text in texts:
        tokens = tokenize_for_domain(text)
        if not tokens:
            continue

        for t in tokens:
            unigram_counter[t] += 1

        for i in range(len(tokens) - 1):
            w1, w2 = tokens[i], tokens[i+1]
            if not is_valid_bigram(w1, w2):
                continue
            bg = f"{w1} {w2}"
            if bg in global_bigrams:
                bigram_counter[bg] += 1

    # 방어: n_top_*가 리스트 등으로 잘못 들어오면 길이로 변환
    if not isinstance(n_top_bigrams, int):
        n_top_bigrams = int(len(n_top_bigrams))
    if not isinstance(n_top_unigrams, int):
        n_top_unigrams = int(len(n_top_unigrams))

    top_bigram_list = [bg for bg, _ in bigram_counter.most_common(n_top_bigrams)]
    top_unigram_list = [w for w, _ in unigram_counter.most_common(n_top_unigrams)]
    combined_keywords = top_bigram_list + top_unigram_list

    return combined_keywords, top_bigram_list, top_unigram_list

def build_cluster_profiles(
    store_clusters: pd.DataFrame,
    reviews_corpus: pd.DataFrame,
    global_bigrams,
    cluster_col: str = "cluster_id",
    store_col: str = "가게이름",
    text_col: str = "리뷰내용",
    top_unigrams: int = 12,
    top_bigrams: int = 3,
) -> pd.DataFrame:
    reviews_with_cluster = reviews_corpus[[store_col, text_col]].merge(
        store_clusters[[store_col, cluster_col]],
        on=store_col,
        how="inner",
    )

    reviews_with_cluster = reviews_with_cluster[
        reviews_with_cluster[cluster_col] != -1
    ].copy()

    rows = []

    for cluster_id, group in reviews_with_cluster.groupby(cluster_col):
        texts = group[text_col].fillna("").tolist()
        n_reviews = len(texts)

        member_stores = sorted(group[store_col].unique().tolist())
        n_stores = len(member_stores)

        # 함수 인자는 n_top_* 이름으로 넘긴다
        cluster_keywords, cluster_bigrams, cluster_unigrams = (
            extract_keywords_and_bigrams_for_cluster(
                texts,
                global_bigrams=global_bigrams,
                n_top_unigrams=top_unigrams,
                n_top_bigrams=top_bigrams,
            )
        )

        rows.append(
            {
                "cluster_id": cluster_id,
                "cluster_keywords": cluster_keywords,
                "cluster_bigrams": cluster_bigrams,
                "cluster_unigrams": cluster_unigrams,
                "member_stores": member_stores,
                "n_stores": n_stores,
                "n_reviews": n_reviews,
            }
        )

    cluster_profiles = pd.DataFrame(rows)
    return cluster_profiles

cluster_profiles = build_cluster_profiles(
    store_clusters=store_clusters,
    reviews_corpus=reviews_corpus,
    global_bigrams=global_bigrams,
    cluster_col="cluster_id",
    store_col="가게이름",
    text_col="리뷰내용",
    top_unigrams=12,
    top_bigrams=3,
)
print(cluster_profiles.head())

   cluster_id                                   cluster_keywords  \
0           0  [쌀국수 맛있, 전골 맛있, 맛있 쌀국수, 맛있, 분위기, 인하대, 친절, 사장, ...   
1           1  [샌드위치 맛있, 샐러드 맛있, 초밥 맛있, 맛있, 친절, 신선, 사장, 샌드위치,...   
2           2  [곱창 맛있, 맛있 곱창, 인하대 곱창, 맛있, 곱창, 인하대, 친절, 사장, 김치...   
3           3  [라멘 맛있, 국물 맛있, 인하대 라멘, 맛있, 친절, 라멘, 인하대, 사장, 국물...   
4           4  [떡볶이 맛있, 쫄면 맛있, 맛있 떡볶이, 맛있, 떡볶이, 쫄면, 친절, 가격, 국...   

            cluster_bigrams  \
0   [쌀국수 맛있, 전골 맛있, 맛있 쌀국수]   
1  [샌드위치 맛있, 샐러드 맛있, 초밥 맛있]   
2    [곱창 맛있, 맛있 곱창, 인하대 곱창]   
3    [라멘 맛있, 국물 맛있, 인하대 라멘]   
4   [떡볶이 맛있, 쫄면 맛있, 맛있 떡볶이]   

                                    cluster_unigrams  \
0  [맛있, 분위기, 인하대, 친절, 사장, 쌀국수, 친구, 양도, 전골, 직원, 가격...   
1  [맛있, 친절, 신선, 사장, 샌드위치, 샐러드, 깔끔, 양도, 인하대, 초밥, 가...   
2  [맛있, 곱창, 인하대, 친절, 사장, 김치, 국수, 돼지, 양도, 가격, 쫄깃, ...   
3  [맛있, 친절, 라멘, 인하대, 사장, 국물, 깔끔, 고기, 신선, 양도, 직원, 가격]   
4  [맛있, 떡볶이, 쫄면, 친절, 가격, 국물, 양도, 인하대, 맛나, 사장, 저렴,...   

                                       member_stores  n_stores  n_r

In [1]:
cluster_profiles

NameError: name 'cluster_profiles' is not defined

In [ ]:
# %% 군집화 추천 대상 가게
import pandas as pd
import json

store_col = "가게이름"

# 1) 클러스터가 할당된 가게만 남기기 (cluster_id != -1)
clustered_stores = store_clusters[store_clusters["cluster_id"] != -1].copy()

# 2) 가게별 키워드 + cluster_id 머지
store_profile_clustered = store_profiles.merge(
    clustered_stores[[store_col, "cluster_id"]],
    on=store_col,
    how="inner"        # 클러스터 없는 가게 자동 제거
)

# 3) 군집별 키워드(cluster_bigrams / cluster_unigrams) 머지
store_profile_clustered = store_profile_clustered.merge(
    cluster_profiles[["cluster_id", "cluster_bigrams", "cluster_unigrams"]],
    on="cluster_id",
    how="left"
)

# 4) 리스트 컬럼을 CSV에 쓰기 좋게 문자열로 변환 (JSON 형식 예시)
list_cols = ["bigrams", "unigrams", "cluster_bigrams", "cluster_unigrams"]

for col in list_cols:
    store_profile_clustered[col] = store_profile_clustered[col].apply(
        lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (list, tuple)) else x
    )

# 네이버 리뷰에 붙이기
naver_with_features = naver_scored.merge(
    store_profile_clustered[
        [store_col, "bigrams", "unigrams", "cluster_id", "cluster_bigrams", "cluster_unigrams"]
    ],
    on=store_col,
    how="inner"   # 클러스터 없는 가게 리뷰는 제거
)

# 카카오 리뷰에 붙이기
kakao_with_features = kakao_scored.merge(
    store_profile_clustered[
        [store_col, "bigrams", "unigrams", "cluster_id", "cluster_bigrams", "cluster_unigrams"]
    ],
    on=store_col,
    how="inner"
)

naver_reviews_final = naver_with_features.rename(columns={
    "bigrams": "store_bigrams",
    "unigrams": "store_unigrams",
    "cluster_bigrams": "group_bigrams",
    "cluster_unigrams": "group_unigrams",
})
print(naver_reviews_final.head())
kakao_reviews_final = kakao_with_features.rename(columns={
    "bigrams": "store_bigrams",
    "unigrams": "store_unigrams",
    "cluster_bigrams": "group_bigrams",
    "cluster_unigrams": "group_unigrams",
})
print(kakao_reviews_final.head())
naver_reviews_final.to_csv(DATA_ROOT / "Naver_Reviews_Final.csv", index=False)
kakao_reviews_final.to_csv(DATA_ROOT / "Kakao_Reviews_Final.csv", index=False)

                                              도입_URL         가게이름    카테고리  \
0  https://map.naver.com/p/entry/place/1392141383...  효자동솥뚜껑 인하대점  돼지고기구이   
1  https://map.naver.com/p/entry/place/1392141383...  효자동솥뚜껑 인하대점  돼지고기구이   
2  https://map.naver.com/p/entry/place/1392141383...  효자동솥뚜껑 인하대점  돼지고기구이   
3  https://map.naver.com/p/entry/place/1392141383...  효자동솥뚜껑 인하대점  돼지고기구이   
4  https://map.naver.com/p/entry/place/1392141383...  효자동솥뚜껑 인하대점  돼지고기구이   

  전체평점  방문자리뷰         리뷰작성자  리뷰작성수  \
0  미공개  608.0       Thumbee   14.0   
1  미공개  608.0       ysa****   18.0   
2  미공개  608.0  yoonyoon1682   32.0   
3  미공개  608.0      Bj990803    7.0   
4  미공개  608.0          곰탱탱스   47.0   

                                                리뷰내용        방문시간  \
0  항강 지나다니던 인하대 골목에서 여러번 지나치다가\n드디어 오게됫어요!\n고기들이 ...  2025-10-31   
1  😋🔥 효자동 솥뚜껑삼겹살 한판 먹고 왔습니다!\n두툼한 삼겹살에 김치, 숙주, 버섯...  2025-10-26   
2  생긴지 얼마 안된 곳인가?싶게 매장이 깨끗해요(화장실도)\n직원분들이 진짜 진짜 친...  2025-10-25   
3  고기도 부드럽고 반찬도 깔끔해서 너무 잘 먹었어요!\n직원분